# Hodina 9: Mini projekt - Návrh jednoduchého projektu s AI

## Obsah hodiny
- Plánování AI projektu
- Výběr vhodných nástrojů
- Implementace chatbota s transformery
- Vytvoření AI asistenta pro studenty
- Deployment s Gradio a Ollama

---

## Instalace potřebných knihoven

In [ ]:
# Instalace knihoven pro projekt
!pip install transformers torch gradio datasets ollama huggingface_hub -q
!pip install langchain chromadb sentence-transformers -q
!pip install streamlit plotly pandas -q

import torch
import gradio as gr
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import datetime
import os

print("✅ Knihovny nainstalovány!")
print(f"PyTorch verze: {torch.__version__}")
print(f"CUDA dostupná: {torch.cuda.is_available()}")

## 1. Plánování projektu

### 🎯 Cíl projektu: AI Studijní Asistent

Vytvoříme inteligentního asistenta, který pomůže studentům s učením:
- 📚 Odpovídání na otázky
- 📝 Generování studijních materiálů
- 🧠 Vytváření kvízů
- 📊 Sledování pokroku
- 💬 Konverzační rozhraní

In [ ]:
# Definice struktury projektu
class ProjectPlanner:
    def __init__(self, project_name):
        self.project_name = project_name
        self.components = []
        self.timeline = []
        self.tech_stack = []
        
    def add_component(self, name, description, priority):
        self.components.append({
            'name': name,
            'description': description,
            'priority': priority,
            'status': 'planned'
        })
    
    def add_tech(self, technology, purpose):
        self.tech_stack.append({
            'technology': technology,
            'purpose': purpose
        })
    
    def visualize_plan(self):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
        
        # Graf 1: Komponenty podle priority
        components_df = pd.DataFrame(self.components)
        priority_counts = components_df['priority'].value_counts()
        
        colors = {'vysoká': '#FF6B6B', 'střední': '#4ECDC4', 'nízká': '#45B7D1'}
        pie_colors = [colors[p] for p in priority_counts.index]
        
        ax1.pie(priority_counts.values, labels=priority_counts.index, 
                colors=pie_colors, autopct='%1.0f%%', startangle=90)
        ax1.set_title('Komponenty podle priority', fontsize=14, fontweight='bold')
        
        # Graf 2: Technologický stack
        tech_names = [t['technology'] for t in self.tech_stack]
        y_positions = np.arange(len(tech_names))
        
        ax2.barh(y_positions, [1]*len(tech_names), color='#96CEB4')
        ax2.set_yticks(y_positions)
        ax2.set_yticklabels(tech_names)
        ax2.set_xlabel('Využití v projektu')
        ax2.set_title('Technologický stack', fontsize=14, fontweight='bold')
        ax2.set_xlim(0, 1.2)
        
        # Přidání účelu technologií
        for i, tech in enumerate(self.tech_stack):
            ax2.text(0.02, i, tech['purpose'], va='center', fontsize=9)
        
        plt.tight_layout()
        plt.show()

# Vytvoření plánu projektu
planner = ProjectPlanner("AI Studijní Asistent")

# Přidání komponent
planner.add_component("Chatbot Interface", "Konverzační rozhraní pro studenty", "vysoká")
planner.add_component("Question Answering", "Odpovídání na studijní otázky", "vysoká")
planner.add_component("Study Material Generator", "Generování poznámek a shrnutí", "vysoká")
planner.add_component("Quiz Creator", "Automatické vytváření testů", "střední")
planner.add_component("Progress Tracker", "Sledování pokroku studenta", "střední")
planner.add_component("Voice Assistant", "Hlasové ovládání", "nízká")
planner.add_component("PDF Analyzer", "Analýza studijních materiálů", "střední")

# Přidání technologií
planner.add_tech("Transformers (Hugging Face)", "NLP modely pro generování textu")
planner.add_tech("Gradio", "Webové uživatelské rozhraní")
planner.add_tech("LangChain", "Orchestrace LLM workflows")
planner.add_tech("ChromaDB", "Vektorová databáze pro RAG")
planner.add_tech("Ollama", "Lokální LLM deployment")
planner.add_tech("PyTorch", "Deep learning framework")

print("📋 PLÁN PROJEKTU: AI Studijní Asistent\n")
print("Komponenty projektu:")
for comp in planner.components:
    emoji = "🔴" if comp['priority'] == 'vysoká' else "🟡" if comp['priority'] == 'střední' else "🔵"
    print(f"{emoji} {comp['name']}: {comp['description']}")

print("\nTehnologický stack:")
for tech in planner.tech_stack:
    print(f"  • {tech['technology']}: {tech['purpose']}")

planner.visualize_plan()

## 2. Implementace AI Chatbota s Transformery

In [ ]:
# Vytvoření AI chatbota pomocí transformerů
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

class StudyAssistantBot:
    def __init__(self, model_name="microsoft/DialoGPT-small"):
        print(f"🤖 Inicializuji AI asistenta s modelem {model_name}...")
        
        # Načtení modelu a tokenizeru
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        
        # Nastavení pad tokenu
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Historie konverzace
        self.conversation_history = []
        self.chat_history_ids = None
        
        # Studijní kontext
        self.study_context = {
            'subject': None,
            'topics': [],
            'questions_asked': 0,
            'correct_answers': 0
        }
        
        print("✅ AI asistent připraven!")
    
    def set_study_context(self, subject, topics):
        """Nastavení studijního kontextu"""
        self.study_context['subject'] = subject
        self.study_context['topics'] = topics
        
    def generate_response(self, user_input, max_length=200):
        """Generování odpovědi pomocí transformeru"""
        # Přidání vstupu do historie
        self.conversation_history.append(f"Student: {user_input}")
        
        # Tokenizace vstupu
        new_user_input_ids = self.tokenizer.encode(
            user_input + self.tokenizer.eos_token, 
            return_tensors='pt'
        )
        
        # Připojení k historii chatu
        if self.chat_history_ids is not None:
            bot_input_ids = torch.cat([self.chat_history_ids, new_user_input_ids], dim=-1)
        else:
            bot_input_ids = new_user_input_ids
        
        # Omezení délky historie
        if bot_input_ids.shape[1] > 1000:
            bot_input_ids = bot_input_ids[:, -1000:]
        
        # Generování odpovědi
        with torch.no_grad():
            chat_history_ids = self.model.generate(
                bot_input_ids,
                max_length=bot_input_ids.shape[1] + max_length,
                pad_token_id=self.tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.8,
                top_p=0.9,
                no_repeat_ngram_size=3
            )
        
        # Dekódování odpovědi
        response = self.tokenizer.decode(
            chat_history_ids[:, bot_input_ids.shape[-1]:][0], 
            skip_special_tokens=True
        )
        
        # Aktualizace historie
        self.chat_history_ids = chat_history_ids
        self.conversation_history.append(f"Asistent: {response}")
        
        # Přidání studijního kontextu
        if self.study_context['subject']:
            response = self._enhance_with_study_context(user_input, response)
        
        return response
    
    def _enhance_with_study_context(self, question, response):
        """Vylepšení odpovědi podle studijního kontextu"""
        # Jednoduchá logika pro vylepšení odpovědí
        question_lower = question.lower()
        
        if "co je" in question_lower or "vysvětli" in question_lower:
            response += f"\n\n💡 Tip: Při studiu {self.study_context['subject']} je důležité pochopit základní koncepty."
        
        elif "jak" in question_lower:
            response += "\n\n📝 Doporučuji si udělat poznámky a procvičit na příkladech."
        
        elif "proč" in question_lower:
            response += "\n\n🔍 Zkuste se zamyslet nad praktickými aplikacemi tohoto konceptu."
        
        return response
    
    def generate_quiz_question(self, topic):
        """Generování kvízové otázky"""
        prompt = f"Vytvoř jednu otázku s možnostmi a, b, c, d na téma {topic}:"
        
        # Simulace generování otázky (v reálné aplikaci by se použil specializovaný model)
        questions = [
            {
                "question": f"Co je hlavní charakteristika {topic}?",
                "options": [
                    "a) První možnost",
                    "b) Druhá možnost",
                    "c) Třetí možnost",
                    "d) Čtvrtá možnost"
                ],
                "correct": "a"
            },
            {
                "question": f"Který z následujících výroků o {topic} je pravdivý?",
                "options": [
                    "a) Výrok 1",
                    "b) Výrok 2",
                    "c) Výrok 3",
                    "d) Výrok 4"
                ],
                "correct": "b"
            }
        ]
        
        return np.random.choice(questions)
    
    def get_study_summary(self):
        """Získání shrnutí studia"""
        if self.study_context['questions_asked'] == 0:
            return "Zatím jste nezačali studovat."
        
        accuracy = (self.study_context['correct_answers'] / 
                   self.study_context['questions_asked'] * 100)
        
        summary = f"""
        📊 **Shrnutí vašeho studia**
        
        Předmět: {self.study_context['subject']}
        Témata: {', '.join(self.study_context['topics'])}
        Zodpovězené otázky: {self.study_context['questions_asked']}
        Správné odpovědi: {self.study_context['correct_answers']}
        Úspěšnost: {accuracy:.1f}%
        
        {self._get_performance_feedback(accuracy)}
        """
        
        return summary
    
    def _get_performance_feedback(self, accuracy):
        """Zpětná vazba podle výkonu"""
        if accuracy >= 90:
            return "🌟 Výborně! Máte skvělé znalosti!"
        elif accuracy >= 70:
            return "👍 Dobře! Ještě trochu procvičit a budete expert!"
        elif accuracy >= 50:
            return "📚 Průměr. Doporučuji více procvičovat."
        else:
            return "💪 Nevzdávejte se! Každý začínal od nuly."

# Vytvoření instance asistenta
print("🎓 VYTVÁŘENÍ AI STUDIJNÍHO ASISTENTA\n")
assistant = StudyAssistantBot()

# Nastavení studijního kontextu
assistant.set_study_context("Umělá inteligence", ["Neural Networks", "Machine Learning", "Deep Learning"])

# Test konverzace
print("\n💬 Ukázka konverzace:\n")

test_questions = [
    "Ahoj, jsem student a potřebuji pomoc s učením.",
    "Co je to neuronová síť?",
    "Jak funguje machine learning?"
]

for question in test_questions:
    print(f"👤 Student: {question}")
    response = assistant.generate_response(question)
    print(f"🤖 Asistent: {response}\n")

# Generování kvízové otázky
print("\n📝 Ukázka kvízové otázky:")
quiz = assistant.generate_quiz_question("neuronové sítě")
print(f"\nOtázka: {quiz['question']}")
for option in quiz['options']:
    print(f"  {option}")
print(f"\n(Správná odpověď: {quiz['correct']})")

## 3. RAG (Retrieval-Augmented Generation) Systém

In [ ]:
# Implementace RAG systému pro přesné odpovědi
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import numpy as np

class StudyRAGSystem:
    def __init__(self):
        print("📚 Inicializuji RAG systém pro studijní materiály...")
        
        # Inicializace embedding modelu
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
        
        # Inicializace ChromaDB
        self.chroma_client = chromadb.Client(Settings(
            anonymized_telemetry=False,
            persist_directory="./study_db"
        ))
        
        # Vytvoření kolekce
        try:
            self.collection = self.chroma_client.create_collection(
                name="study_materials",
                metadata={"hnsw:space": "cosine"}
            )
        except:
            self.collection = self.chroma_client.get_collection("study_materials")
        
        print("✅ RAG systém připraven!")
    
    def add_study_material(self, text, metadata=None):
        """Přidání studijního materiálu do databáze"""
        # Rozdělení textu na chunks
        chunks = self._split_text(text, chunk_size=200)
        
        # Vytvoření embeddings
        embeddings = self.embedder.encode(chunks)
        
        # Přidání do databáze
        for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
            self.collection.add(
                embeddings=[embedding.tolist()],
                documents=[chunk],
                metadatas=[metadata or {}],
                ids=[f"chunk_{len(self.collection.get()['ids'])}_{i}"]
            )
        
        print(f"✅ Přidáno {len(chunks)} částí textu do databáze")
    
    def _split_text(self, text, chunk_size=200):
        """Rozdělení textu na menší části"""
        words = text.split()
        chunks = []
        
        for i in range(0, len(words), chunk_size):
            chunk = ' '.join(words[i:i+chunk_size])
            chunks.append(chunk)
        
        return chunks
    
    def search(self, query, n_results=3):
        """Vyhledání relevantních materiálů"""
        # Vytvoření embedding pro dotaz
        query_embedding = self.embedder.encode([query])[0]
        
        # Vyhledání v databázi
        results = self.collection.query(
            query_embeddings=[query_embedding.tolist()],
            n_results=n_results
        )
        
        return results
    
    def generate_answer(self, query, context_docs):
        """Generování odpovědi na základě kontextu"""
        # Spojení relevantních dokumentů
        context = "\n\n".join(context_docs)
        
        # Jednoduchá šablona pro odpověď
        answer_template = f"""
        Na základě dostupných materiálů:
        
        {context}
        
        Odpověď na otázku "{query}":
        """
        
        # V reálné aplikaci by se použil LLM pro generování odpovědi
        # Pro demo účely vrátíme strukturovanou odpověď
        return self._simple_answer_generation(query, context)
    
    def _simple_answer_generation(self, query, context):
        """Jednoduchá generace odpovědi (simulace)"""
        query_lower = query.lower()
        
        if "co je" in query_lower:
            return f"Na základě studijních materiálů: {context[:200]}..."
        elif "jak" in query_lower:
            return f"Postup podle materiálů: {context[:200]}..."
        elif "proč" in query_lower:
            return f"Důvod je následující: {context[:200]}..."
        else:
            return f"Relevantní informace: {context[:200]}..."

# Vytvoření RAG systému
rag_system = StudyRAGSystem()

# Přidání studijních materiálů
study_materials = [
    """
    Neuronové sítě jsou výpočetní modely inspirované biologickým mozkem. 
    Skládají se z vrstev neuronů, které jsou vzájemně propojeny vahami. 
    Každý neuron přijímá vstupy, aplikuje na ně váhy, sečte je a 
    výsledek prožene aktivační funkcí. Učení probíhá pomocí algoritmu 
    zpětné propagace, který upravuje váhy na základě chyby predikce.
    """,
    
    """
    Strojové učení je podoblast umělé inteligence, která umožňuje 
    počítačům učit se z dat bez explicitního programování. Existují 
    tři hlavní typy: učení s učitelem (supervised learning), učení 
    bez učitele (unsupervised learning) a posilované učení 
    (reinforcement learning). Každý typ má své specifické použití.
    """,
    
    """
    Deep learning je specializovaná oblast strojového učení využívající 
    hluboké neuronové sítě s mnoha vrstvami. Tyto sítě dokáží 
    automaticky extrahovat hierarchické features z dat. Nejznámější 
    architektury zahrnují CNN pro zpracování obrazu, RNN pro 
    sekvenční data a Transformery pro NLP úlohy.
    """
]

print("\n📥 Přidávám studijní materiály do databáze...")
for i, material in enumerate(study_materials):
    rag_system.add_study_material(
        material, 
        metadata={"topic": ["Neural Networks", "Machine Learning", "Deep Learning"][i]}
    )

# Test vyhledávání
print("\n🔍 Test RAG systému:\n")

test_queries = [
    "Co jsou neuronové sítě?",
    "Jaké jsou typy strojového učení?",
    "Co je deep learning?"
]

for query in test_queries:
    print(f"❓ Dotaz: {query}")
    
    # Vyhledání relevantních dokumentů
    results = rag_system.search(query, n_results=2)
    
    if results['documents'][0]:
        # Generování odpovědi
        answer = rag_system.generate_answer(query, results['documents'][0])
        print(f"💡 Odpověď: {answer}")
    else:
        print("❌ Nenalezeny žádné relevantní materiály")
    
    print("-" * 80)

## 4. Vytvoření interaktivního rozhraní s Gradio

In [ ]:
# Kompletní Gradio aplikace pro AI Studijního Asistenta
import gradio as gr
import json
from datetime import datetime

class StudyAssistantApp:
    def __init__(self):
        self.assistant = StudyAssistantBot()
        self.rag_system = StudyRAGSystem()
        self.study_history = []
        self.current_quiz = None
        
    def chat_interface(self, message, history):
        """Chatovací rozhraní"""
        # Získání odpovědi od asistenta
        response = self.assistant.generate_response(message)
        
        # Uložení do historie
        self.study_history.append({
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'question': message,
            'answer': response
        })
        
        return response
    
    def rag_search(self, query):
        """RAG vyhledávání"""
        results = self.rag_system.search(query)
        
        if results['documents'][0]:
            answer = self.rag_system.generate_answer(query, results['documents'][0])
            sources = "\n\n📚 Zdroje:\n" + "\n".join([f"- {doc[:100]}..." for doc in results['documents'][0]])
            return answer + sources
        else:
            return "Nenalezeny žádné relevantní materiály. Zkuste přeformulovat dotaz."
    
    def generate_quiz(self, topic, difficulty):
        """Generování kvízu"""
        # Generování otázek podle obtížnosti
        num_questions = {"Lehká": 3, "Střední": 5, "Těžká": 7}[difficulty]
        
        quiz_questions = []
        for i in range(num_questions):
            q = self.assistant.generate_quiz_question(topic)
            quiz_questions.append(q)
        
        self.current_quiz = {
            'topic': topic,
            'difficulty': difficulty,
            'questions': quiz_questions,
            'user_answers': [],
            'start_time': datetime.now()
        }
        
        # Formátování kvízu pro zobrazení
        quiz_text = f"📝 **Kvíz: {topic}** (Obtížnost: {difficulty})\n\n"
        
        for i, q in enumerate(quiz_questions, 1):
            quiz_text += f"**Otázka {i}:** {q['question']}\n"
            for option in q['options']:
                quiz_text += f"  {option}\n"
            quiz_text += "\n"
        
        return quiz_text
    
    def submit_quiz(self, answers):
        """Vyhodnocení kvízu"""
        if not self.current_quiz:
            return "Nejprve vygenerujte kvíz!"
        
        # Parsování odpovědí
        user_answers = answers.strip().split(',')
        correct_count = 0
        
        results = f"📊 **Výsledky kvízu**\n\n"
        
        for i, (question, user_answer) in enumerate(zip(self.current_quiz['questions'], user_answers)):
            user_answer = user_answer.strip().lower()
            is_correct = user_answer == question['correct']
            
            if is_correct:
                correct_count += 1
                results += f"✅ Otázka {i+1}: Správně!\n"
            else:
                results += f"❌ Otázka {i+1}: Špatně (správná odpověď: {question['correct']})\n"
        
        # Celkové hodnocení
        total = len(self.current_quiz['questions'])
        percentage = (correct_count / total) * 100
        
        results += f"\n**Celkové skóre: {correct_count}/{total} ({percentage:.0f}%)**\n"
        
        # Aktualizace statistik
        self.assistant.study_context['questions_asked'] += total
        self.assistant.study_context['correct_answers'] += correct_count
        
        # Doporučení
        if percentage >= 80:
            results += "\n🎉 Výborně! Pokračujte na další téma."
        elif percentage >= 60:
            results += "\n👍 Dobře! Ještě trochu procvičit problematické oblasti."
        else:
            results += "\n📚 Doporučuji znovu prostudovat toto téma."
        
        return results
    
    def add_study_material(self, file):
        """Přidání studijního materiálu"""
        if file is None:
            return "Prosím nahrajte soubor."
        
        try:
            # Čtení souboru
            content = file.read().decode('utf-8')
            
            # Přidání do RAG systému
            self.rag_system.add_study_material(content)
            
            return f"✅ Materiál úspěšně přidán! Délka: {len(content)} znaků"
        except Exception as e:
            return f"❌ Chyba při zpracování souboru: {str(e)}"
    
    def get_study_progress(self):
        """Získání studijního pokroku"""
        return self.assistant.get_study_summary()

# Vytvoření aplikace
app = StudyAssistantApp()

# Vytvoření Gradio interface
with gr.Blocks(title="AI Studijní Asistent", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🎓 AI Studijní Asistent
    
    Váš osobní AI asistent pro efektivní učení!
    """)
    
    with gr.Tabs():
        # Tab 1: Chat
        with gr.Tab("💬 Chat s Asistentem"):
            chatbot = gr.ChatInterface(
                fn=app.chat_interface,
                title="Zeptejte se na cokoliv!",
                examples=[
                    "Co je to neuronová síť?",
                    "Jak se učit efektivně?",
                    "Vysvětli mi gradient descent",
                    "Jaký je rozdíl mezi AI a ML?"
                ],
                retry_btn="🔄 Zkusit znovu",
                undo_btn="↩️ Vrátit",
                clear_btn="🗑️ Vymazat"
            )
        
        # Tab 2: RAG Vyhledávání
        with gr.Tab("🔍 Vyhledávání v materiálech"):
            with gr.Row():
                search_input = gr.Textbox(
                    label="Zadejte dotaz",
                    placeholder="Např. 'Jak funguje backpropagation?'",
                    lines=2
                )
                search_btn = gr.Button("🔍 Vyhledat", variant="primary")
            
            search_output = gr.Textbox(
                label="Výsledky vyhledávání",
                lines=10,
                interactive=False
            )
            
            search_btn.click(
                fn=app.rag_search,
                inputs=search_input,
                outputs=search_output
            )
        
        # Tab 3: Kvízy
        with gr.Tab("📝 Kvízy a testy"):
            with gr.Row():
                quiz_topic = gr.Textbox(
                    label="Téma kvízu",
                    placeholder="Např. 'Neuronové sítě'"
                )
                quiz_difficulty = gr.Radio(
                    ["Lehká", "Střední", "Těžká"],
                    label="Obtížnost",
                    value="Střední"
                )
                generate_quiz_btn = gr.Button("🎲 Generovat kvíz", variant="primary")
            
            quiz_display = gr.Textbox(
                label="Kvíz",
                lines=15,
                interactive=False
            )
            
            with gr.Row():
                quiz_answers = gr.Textbox(
                    label="Vaše odpovědi (oddělené čárkou)",
                    placeholder="a, b, c, d, a"
                )
                submit_quiz_btn = gr.Button("📊 Vyhodnotit", variant="secondary")
            
            quiz_results = gr.Textbox(
                label="Výsledky",
                lines=10,
                interactive=False
            )
            
            generate_quiz_btn.click(
                fn=app.generate_quiz,
                inputs=[quiz_topic, quiz_difficulty],
                outputs=quiz_display
            )
            
            submit_quiz_btn.click(
                fn=app.submit_quiz,
                inputs=quiz_answers,
                outputs=quiz_results
            )
        
        # Tab 4: Správa materiálů
        with gr.Tab("📚 Správa materiálů"):
            gr.Markdown("### Nahrajte studijní materiály")
            
            file_upload = gr.File(
                label="Vyberte textový soubor",
                file_types=[".txt", ".md"],
                type="binary"
            )
            
            upload_btn = gr.Button("📤 Nahrát materiál", variant="primary")
            upload_status = gr.Textbox(label="Status", interactive=False)
            
            upload_btn.click(
                fn=app.add_study_material,
                inputs=file_upload,
                outputs=upload_status
            )
        
        # Tab 5: Pokrok
        with gr.Tab("📊 Můj pokrok"):
            refresh_btn = gr.Button("🔄 Aktualizovat", variant="primary")
            progress_display = gr.Textbox(
                label="Studijní pokrok",
                lines=15,
                interactive=False
            )
            
            refresh_btn.click(
                fn=app.get_study_progress,
                outputs=progress_display
            )
    
    gr.Markdown("""
    ---
    
    ### 🚀 Tipy pro efektivní studium:
    - Ptejte se na konkrétní otázky
    - Procvičujte pomocí kvízů
    - Nahrajte vlastní studijní materiály
    - Sledujte svůj pokrok
    
    Vytvořeno s ❤️ pomocí AI
    """)

# Spuštění aplikace
print("\n🚀 Spouštím AI Studijního Asistenta...")
demo.launch(share=True)

## 5. Integrace s Ollama pro lokální deployment

In [ ]:
# Skript pro lokální deployment s Ollama
import subprocess
import requests
import json

class OllamaStudyAssistant:
    def __init__(self, model="llama2"):
        self.model = model
        self.base_url = "http://localhost:11434"
        
    def check_ollama_status(self):
        """Kontrola, zda Ollama běží"""
        try:
            response = requests.get(f"{self.base_url}/api/tags")
            if response.status_code == 200:
                print("✅ Ollama server běží")
                return True
            else:
                print("❌ Ollama server neodpovídá")
                return False
        except:
            print("❌ Nelze se připojit k Ollama serveru")
            return False
    
    def list_models(self):
        """Seznam dostupných modelů"""
        try:
            response = requests.get(f"{self.base_url}/api/tags")
            if response.status_code == 200:
                models = response.json().get('models', [])
                print("📋 Dostupné modely:")
                for model in models:
                    print(f"  - {model['name']} ({model['size']} bytes)")
                return models
            return []
        except Exception as e:
            print(f"Chyba: {e}")
            return []
    
    def generate(self, prompt, context=""):
        """Generování odpovědi pomocí Ollama"""
        try:
            data = {
                "model": self.model,
                "prompt": f"{context}\n\n{prompt}",
                "stream": False
            }
            
            response = requests.post(
                f"{self.base_url}/api/generate",
                json=data
            )
            
            if response.status_code == 200:
                return response.json()['response']
            else:
                return f"Chyba: {response.status_code}"
                
        except Exception as e:
            return f"Chyba při generování: {e}"
    
    def create_study_prompt(self, question, subject):
        """Vytvoření promptu pro studijního asistenta"""
        prompt = f"""
        Jsi AI studijní asistent specializovaný na předmět: {subject}.
        
        Tvým úkolem je:
        1. Odpovídat jasně a srozumitelně
        2. Používat příklady
        3. Strukturovat odpovědi
        4. Být trpělivý a nápomocný
        
        Otázka studenta: {question}
        
        Odpověď:
        """
        
        return prompt

# Vytvoření deployment skriptu
deployment_script = '''
#!/bin/bash

# AI Studijní Asistent - Deployment Script

echo "🚀 Instalace AI Studijního Asistenta"

# 1. Kontrola Ollama
if ! command -v ollama &> /dev/null; then
    echo "📥 Instaluji Ollama..."
    curl -fsSL https://ollama.ai/install.sh | sh
fi

# 2. Stažení modelu
echo "📥 Stahuji model llama2..."
ollama pull llama2

# 3. Spuštění Ollama serveru
echo "🖥️ Spouštím Ollama server..."
ollama serve &

# 4. Vytvoření Python prostředí
echo "🐍 Vytvářím Python prostředí..."
python3 -m venv venv
source venv/bin/activate

# 5. Instalace závislostí
echo "📦 Instaluji závislosti..."
pip install gradio transformers torch ollama chromadb sentence-transformers

# 6. Spuštění aplikace
echo "🎓 Spouštím AI Studijního Asistenta..."
python study_assistant.py
'''

# Uložení deployment skriptu
with open('deploy_assistant.sh', 'w') as f:
    f.write(deployment_script)

print("📄 Deployment skript vytvořen: deploy_assistant.sh")

# Test Ollama (pokud je dostupná)
print("\n🔍 Test Ollama integrace:")
ollama_assistant = OllamaStudyAssistant()

if ollama_assistant.check_ollama_status():
    ollama_assistant.list_models()
    
    # Test generování
    test_prompt = ollama_assistant.create_study_prompt(
        "Co je to gradient descent?",
        "Strojové učení"
    )
    
    print("\n🤖 Testovací odpověď:")
    # response = ollama_assistant.generate(test_prompt)
    # print(response)
else:
    print("\n💡 Pro lokální deployment:")
    print("1. Nainstalujte Ollama: https://ollama.ai")
    print("2. Stáhněte model: ollama pull llama2")
    print("3. Spusťte server: ollama serve")
    print("4. Spusťte skript: bash deploy_assistant.sh")

## 6. Shrnutí projektu a další kroky

In [ ]:
# Vytvoření dokumentace projektu
project_documentation = """
# 🎓 AI Studijní Asistent - Dokumentace

## 📋 Přehled projektu

AI Studijní Asistent je komplexní aplikace využívající nejmodernější technologie 
umělé inteligence pro podporu studentů při učení.

## 🛠️ Technologie

- **Transformers**: Hugging Face modely pro NLP
- **Gradio**: Interaktivní webové rozhraní
- **ChromaDB**: Vektorová databáze pro RAG
- **Ollama**: Lokální deployment LLM
- **PyTorch**: Deep learning framework

## 🚀 Funkce

1. **Chatbot**: Konverzační asistent pro zodpovídání otázek
2. **RAG systém**: Vyhledávání v studijních materiálech
3. **Generátor kvízů**: Automatické vytváření testů
4. **Sledování pokroku**: Statistiky a analýzy učení
5. **Správa materiálů**: Upload a organizace studijních zdrojů

## 📦 Instalace

```bash
# Klonování repozitáře
git clone https://github.com/your-username/ai-study-assistant.git
cd ai-study-assistant

# Instalace závislostí
pip install -r requirements.txt

# Spuštění aplikace
python app.py
```

## 🎯 Použití

1. Spusťte aplikaci
2. Otevřete webový prohlížeč na http://localhost:7860
3. Začněte chatovat s asistentem
4. Nahrajte studijní materiály
5. Generujte kvízy a sledujte pokrok

## 🔧 Konfigurace

Upravte `config.yaml` pro:
- Výběr modelu
- Nastavení RAG parametrů
- UI přizpůsobení

## 📈 Budoucí vylepšení

- [ ] Podpora více jazyků
- [ ] Hlasové ovládání
- [ ] Export studijních materiálů
- [ ] Mobilní aplikace
- [ ] Integrace s LMS systémy

## 👥 Přispívání

Příspěvky jsou vítány! Prosím:
1. Forkněte repozitář
2. Vytvořte feature branch
3. Commitujte změny
4. Pushněte branch
5. Otevřete Pull Request

## 📄 Licence

MIT License - volně k použití pro vzdělávací účely

## 🙏 Poděkování

- Hugging Face za transformery
- Gradio tým za skvělé UI
- Ollama za lokální LLM
- Všem přispěvatelům
"""

# Uložení dokumentace
with open('README.md', 'w', encoding='utf-8') as f:
    f.write(project_documentation)

print("📄 Dokumentace vytvořena: README.md")

# Vytvoření requirements.txt
requirements = """
transformers>=4.30.0
torch>=2.0.0
gradio>=3.35.0
chromadb>=0.4.0
sentence-transformers>=2.2.0
langchain>=0.0.200
ollama>=0.1.0
pandas>=2.0.0
numpy>=1.24.0
matplotlib>=3.7.0
plotly>=5.14.0
"""

with open('requirements.txt', 'w') as f:
    f.write(requirements)

print("📦 Requirements vytvořeny: requirements.txt")

# Finální shrnutí
print("\n" + "="*60)
print("🎉 PROJEKT ÚSPĚŠNĚ DOKONČEN!")
print("="*60)
print("\n📊 Statistiky projektu:")
print("  - Komponenty: 7")
print("  - Technologie: 6")
print("  - Řádky kódu: ~1000")
print("  - Funkce: Chat, RAG, Kvízy, Progress tracking")
print("\n🚀 Další kroky:")
print("  1. Otestovat s reálnými studenty")
print("  2. Vylepšit UI/UX")
print("  3. Přidat více modelů")
print("  4. Implementovat mobilní verzi")
print("  5. Integrovat s učebními platformami")
print("\n💡 Tip: Zkuste aplikaci rozšířit o vlastní funkce!")

## 7. Cvičení a úkoly

### Úkol 1: Rozšíření funkcionalit

In [ ]:
# Úkol: Přidejte novou funkci do AI asistenta
class StudyPlannerExtension:
    def __init__(self):
        self.study_plans = {}
        
    def create_study_plan(self, subject, available_hours, deadline):
        """
        TODO: Implementujte vytváření studijního plánu
        - Rozdělte téma na menší části
        - Alokujte čas pro každou část
        - Vytvořte harmonogram
        """
        pass
    
    def track_progress(self, plan_id, completed_section):
        """
        TODO: Sledujte pokrok ve studijním plánu
        """
        pass
    
    def get_recommendations(self, student_performance):
        """
        TODO: Generujte personalizovaná doporučení
        """
        pass

# Implementujte rozšíření

## 8. Shrnutí

### Co jsme vytvořili:
- ✅ Kompletní AI studijní asistent
- ✅ Chatbot s transformery
- ✅ RAG systém pro přesné odpovědi
- ✅ Generátor kvízů
- ✅ Interaktivní Gradio rozhraní
- ✅ Integrace s Ollama

### Naučené dovednosti:
1. **Plánování AI projektu**
2. **Práce s transformery**
3. **Implementace RAG**
4. **Vytváření UI s Gradio**
5. **Deployment s Ollama**

### Příště:
V další hodině shrneme vše, co jsme se naučili!

## 9. Domácí úkol

1. **Vylepšete chatbota**:
   - Přidejte podporu pro více předmětů
   - Implementujte paměť konverzace
   - Vylepšete generování odpovědí

2. **Rozšiřte RAG systém**:
   - Přidejte podporu PDF souborů
   - Implementujte hodnocení relevance
   - Vytvořte vizualizaci výsledků

3. **Vytvořte mobilní verzi**:
   - Použijte Gradio mobile support
   - Optimalizujte UI pro mobily
   - Přidejte offline podporu

4. **Napište dokumentaci**:
   - Uživatelský manuál
   - API dokumentace
   - Instalační video návod

---

**🏆 Výzva**: Publikujte váš projekt na GitHub a sdílejte s komunitou!

### Hodina 9 — Mini projekt: krátké shrnutí (ELI10)

Dnes si vybereme jednoduchý problém a vytvoříme malý prototyp. Projekt by měl být dost malý, aby ho bylo možné dokončit během jedné nebo dvou hodin. Doporučení: klasifikace malého datasetu (např. Iris) s jednoduchým uživatelským rozhraním přes Gradio, nebo hra, kterou ovládá jednoduchý algoritmus.

Níže je krátká kostra projektu, kterou můžete zkopírovat a upravit.

In [ ]:
# Mini-project scaffold: Iris classifier (very small prototype)
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=42)
clf = RandomForestClassifier(n_estimators=10, random_state=42)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
acc = accuracy_score(y_test, pred)
print('Accuracy on Iris test set:', acc)
assert acc >= 0.8  # simple sanity check — many runs will satisfy this

# Next steps: wrap `clf.predict` into a Gradio interface or save the model for use in the project.

Úkoly pro mini projekt:

1) Zvolte problém (klasifikace, jednoduchá hra nebo detekce) a napište krátký popis cíle projektu.
2) Implementujte prototyp (př. Iris classifier nebo jednoduché Gradio UI pro hru).
3) Připravte krátkou prezentaci (2–3 slidy nebo 5 minut), kde vysvětlíte, jak problém řešíte a jaké metody jste použili.
4) Volitelně: nasdílejte Gradio demo online pro snadné testování ostatními.